<a href="https://colab.research.google.com/github/wchen928/TDS-recomsys/blob/master/EA_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/TDS_Tech"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Copy of New authors.gsheet'	'Progress Report'
 EA.gsheet			'Real-time Dashboard.gdoc'
 EA_recom.ipynb			'Recommendation System Plan.gdoc'
"Linda's attempt to dashboard"	 Timeline.gsheet


In [ ]:
!pip install xlrd
!pip install psycopg2
!pip install --upgrade gspread

Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [ ]:
import pandas as pd
import numpy  as np

In [ ]:
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('EA').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

import pandas as pd
df = pd.DataFrame.from_records(rows)

## Data Cleaning

In [ ]:
df.head()

,0,1,2,3
0,Names,Keywords,LinkedIn,Current Title
1,Lowri Williams,"Python, NLP",https://www.linkedin.com/in/lowriwilliams/,Research Data Scientist at Cardiff University
2,Sophie Mann,"cognitive science/neuroscience, healthcare, ed...",https://www.linkedin.com/in/sophie-maya-mann/,Data Manager at Tennessee Department of Education
3,Jingles,in heath sector (eye),https://www.linkedin.com/in/jingles/,PhD Student | Alibaba Group | Healthcare (Eye ...
4,Gitansh Khirbat,"NLP, Statistics, Optimizations",https://www.linkedin.com/in/gkhirbat/,Machine Learning Engineer | Applied Data Science


In [ ]:
#turn the first row into column
new_header = df.iloc[0] #grab the first row for the header
df = df.iloc[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df.head()

,Names,Keywords,LinkedIn,Current Title
1,Lowri Williams,"Python, NLP",https://www.linkedin.com/in/lowriwilliams/,Research Data Scientist at Cardiff University
2,Sophie Mann,"cognitive science/neuroscience, healthcare, ed...",https://www.linkedin.com/in/sophie-maya-mann/,Data Manager at Tennessee Department of Education
3,Jingles,in heath sector (eye),https://www.linkedin.com/in/jingles/,PhD Student | Alibaba Group | Healthcare (Eye ...
4,Gitansh Khirbat,"NLP, Statistics, Optimizations",https://www.linkedin.com/in/gkhirbat/,Machine Learning Engineer | Applied Data Science
5,Abdullah Farouk,deep learning,https://www.linkedin.com/in/abdullah-farouk/,


In [ ]:
df['description'] = df['Keywords'] + " " + df['Current Title']

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def clean_text(s):
    #removeNonAscii
    s = "".join(i for i in s if  ord(i)<128)

    #return all lower cases
    s.lower()

    #remove stop wrods
    s = s.split()
    stops = set(stopwords.words("english"))
    text = [w for w in s if not w in stops]
    text = " ".join(text)

    #remove html
    html_pattern = re.compile('<.*?>')
    text = html_pattern.sub(r'', text)

    #remove punctuation
    text = re.sub(r'[^\w\s]'," ",text)
    return text




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Build Word2Vec Model

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [ ]:
# Downloading the Google pretrained Word2Vec Model
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Training our corpus with Google Pretrained Model

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

--2020-09-30 18:37:29--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.45.62
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.45.62|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(0, 0)

In [ ]:
# Generate the average word2vec for the each book description

def word2vez(text):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    vectors = {}

    avgword2vec = None
    count = 0
    for word in text.split():
        if word in google_model.wv.vocab:
            count += 1
            if avgword2vec is None:
                avgword2vec = google_model[word]
            else:
                avgword2vec = avgword2vec + google_model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
    word_embeddings.append(avgword2vec)

    return word_embeddings

## Recomm System

Here I still have the question of wheather using cosine_similarity or Word Mover’s Distance

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Recommending the Top 5 similar books

def recommendations(input, df=df, col='description', m=3):
    
    # vecterizd the input
    input = clean_text(input)
    vector_input = word2vez(input)

    res = {}
    # vecterized all the EA
    df[col] = df[col].apply(clean_text)
    EAs_vectors = [word2vez(EA) for EA in df[col]]

    df['vectors'] = EAs_vectors

    for i, n in enumerate(df['vectors']):
      if n[0] is None:
        df = df.drop([i+1], axis=0)


    # finding cosine similarity for the vectors
    similarity = []
    for n in df['vectors']:
      scores = cosine_similarity(vector_input, n)[0][0]
      similarity.append(scores)
    
    df['similarity'] = similarity


    #
    df = df.sort_values(by=['similarity'], ascending=False)
    res_df = df.loc[:m+1]


    return res_df['Names']


In [ ]:
input = 'In this article, I’ll describe what I believe are some best practices \n
to start a Reinforcement Learning (RL) project. I’ll do this by illustrating \n
some lessons I learned when I replicated Deepmind’s performance on video \n
games. This was a fun side-project I worked on.'

res = recommendations(input=input, df=df, col='description', m=3)

print(res)

#print(res[res['vectors'][0] == None])

#input = clean_text(input)
#vector_input = word2vez(input)

#cosine_similarities = cosine_similarity(vector_input,EAs_vectors[1])

"""
similarity = []
count = 0
for n in range(0, len(EAs_vectors)-1):
  scores = cosine_similarity(vector_input, EAs_vectors[n])[0][0]
  similarity.append(scores)
  count += 1
  print(count, scores)
"""



19    Pier Paolo Ippolito
31             Agni Kumar
4         Gitansh Khirbat
Name: Names, dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


'\nsimilarity = []\ncount = 0\nfor n in range(0, len(EAs_vectors)-1):\n  scores = cosine_similarity(vector_input, EAs_vectors[n])[0][0]\n  similarity.append(scores)\n  count += 1\n  print(count, scores)\n'

## Problem:

Some words like "R", "Python" "ML" are not in the pre-trained Word2Vec model. Thus, there were 5 EAs have the value None.